In [ ]:
%matplotlib inline
from moviepy.editor import VideoFileClip
import numpy as np
import cv2

class LineInfo:
    def __init__(self, slope, center_x, length_sqr, is_valid):
        self.slope = slope
        self.center_x = center_x
        self.length_sqr = length_sqr
        self.is_valid = is_valid

def filterLines(lines):
    lineInfo = []

    # Calculate some additional information for the lines
    for line in lines:
        for x1, y1, x2, y2 in line:
            deltaY = y2 - y1
            deltaX = x2 - x1
            slope = deltaY / deltaX
            centerX = (x1 + x2) / 2
            lengthSqr = deltaX * deltaX + deltaY * deltaY
            lineInfo.append(LineInfo(slope, centerX, lengthSqr, True))

    slopeThreshold = 0.1
    centerXThreshold = 10
    validCount = len(lineInfo)

    # Filter the lines until only two remain
    while validCount > 2:
        similar_lines_found = False
        for line in lineInfo:
            if not line.is_valid:
                continue
            for line_other in lineInfo:
                if line == line_other or not line_other.is_valid:
                    continue

                # Compare slope and delta of center_x to determine if line and line_other are similar
                slopeCriterion = line.slope - line_other.slope < slopeThreshold
                centerXCriterion = abs(line.center_x - line_other.center_x) < centerXThreshold

                if slopeCriterion and centerXCriterion:
                    validCount -= 1
                    similar_lines_found = True
                    # Mark the shorter of the two similar lines as invalid
                    if line.length_sqr < line_other.length_sqr:
                        line.is_valid = False
                        # The current reference line is invalid, so break and choose a new one
                        break
                    else:
                        line_other.is_valid = False

        # Increase the thresholds as long as there are more than two lines
        if not similar_lines_found:
            slopeThreshold *= 2
            centerXThreshold *= 2

    filteredLines = []
    for idx, line in enumerate(lineInfo):
        if line.is_valid:
            filteredLines.append(lines[idx])

    return filteredLines


def filterByColor(img):
    # Filtering by yellow
    red_min = 200
    red_max = 255
    green_min = 180
    green_max = 255
    blue_min = 20
    blue_max = 120

    thresholds = (img[:, :, 0] < red_min) | (img[:, :, 1] < green_min) | (img[:, :, 2] < blue_min) | \
                 (img[:, :, 0] > red_max) | (img[:, :, 1] > green_max) | (img[:, :, 2] > blue_max)

    # Filtering by white
    red_min = 200
    green_min = 200
    blue_min = 200
    thresholds2 = (img[:, :, 0] < red_min) | (img[:, :, 1] < green_min) | (img[:, :, 2] < blue_min)

    filtered_img = np.copy(img)
    filtered_img[thresholds & thresholds2] = [0, 0, 0]
    return filtered_img


def convertToGrayScaleAndBlur(img):
    img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    kernel_size = 5
    gray_blurred_img = cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)
    return gray_blurred_img


def maskImage(img):
    mask = np.zeros_like(img)
    ignore_mask_color = 255
    width = img.shape[1]
    height = img.shape[0]
    vertices = np.array([[(150, height - 60), (width / 2 - 100, 450), (width / 2 + 100, 450), (width - 150, height - 60)]], dtype=np.int32)
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    masked_img = cv2.bitwise_and(img, mask)
    return masked_img


def detectEdges(img):
    edges_img = cv2.Canny(img, 50, 150)
    return edges_img


def detectLines(img, edges_img):
    rho = 2
    theta = np.pi / 180
    threshold = 15
    min_line_length = 30
    max_line_gap = 100

    lines = cv2.HoughLinesP(edges_img, rho, theta, threshold, np.array([]), min_line_length, max_line_gap)
    lines = filterLines(lines)

    line_image = np.zeros_like(img)
    for line in lines:
        for x1, y1, x2, y2 in line:
            cv2.line(line_image, (x1, y1), (x2, y2), (255, 0, 0), 10)

    return line_image


def drawLinesToImage(img, line_image):
    fused_img = cv2.addWeighted(img, 0.8, line_image, 1, 0)
    return fused_img


def processImage(img):
    filtered_img = filterByColor(img)
    gray_blurred_img = convertToGrayScaleAndBlur(filtered_img)
    masked_img = maskImage(gray_blurred_img)
    edges_img = detectEdges(masked_img)
    line_image = detectLines(img, edges_img)
    fused_img = drawLinesToImage(img, line_image)
    return fused_img


clip = VideoFileClip('test.mp4')
modifiedClip = clip.fl_image(processImage)
modifiedClip.write_videofile('test_with_lines.mp4', audio=False)

[MoviePy] >>>> Building video test_with_lines.mp4


[MoviePy] Writing video test_with_lines.mp4


  0%|          | 0/251 [00:00<?, ?it/s]

  1%|          | 2/251 [00:00<00:14, 17.10it/s]

  2%|▏         | 4/251 [00:00<00:14, 16.75it/s]

...

 98%|█████████▊| 246/251 [00:14<00:00, 17.28it/s]

 99%|█████████▉| 248/251 [00:14<00:00, 17.29it/s]

100%|██████████| 251/251 [00:14<00:00, 17.32it/s]

[MoviePy] Done.


[MoviePy] >>>> Video ready: test_with_lines.mp4 

